# Scenemixer
Tool for combining two photographs into a composite image (center + surrounding).

In [1]:
from __future__ import division
import numpy as np
from skimage import io, transform, filters, color
from skimage import img_as_uint, img_as_float
from tools import *
%matplotlib inline

### Settings
Check file extensions: `.jpg` differs from `.jpeg` and differ from `.JPG` (in some operating systems)

In [2]:
image_folder = "./images2/"  # must end with /
# filenames in []
image_in     = ["lapland.jpeg", "sarka.jpeg", "helsinki.jpeg"]  
image_in     = [
"MH1.jpg", "MH2.jpg", "MH3.jpg", "MH4.jpg", "MH5.jpg", "MM1.jpg", "MM2.jpg", "MM3.jpg", "MM4.jpg", "MM5.jpg", 
"MP1.jpg", "MP2.jpg", "MP3.jpg", "MP4.jpg", "MP5.jpg", "MS1.jpg", "MS2.jpg", "MS3.jpg", "MS4.jpg", "MS5.jpg", 
"MT1.jpg", "MT2.jpg", "MT3.jpg", "MT4.jpg", "MT5.jpg", "NB1.jpg", "NB2.jpg", "NB3.jpg", "NB4.jpg", "NB5.jpg", 
"ND1.jpg", "ND2.jpg", "ND3.jpg", "ND4.jpg", "ND5.jpg", "NF1.jpg", "NF2.jpg", "NF3.jpg", "NF4.jpg", "NF5.jpg", 
"NM1.jpg", "NM2.jpg", "NM3.jpg", "NM4.jpg", "NM5.jpg", "NR1.jpg", "NR2.jpg", "NR3.jpg", "NR4.jpg", "NR5.jpg"
]
                                                
image_out    = image_in     # use the same as image_in
new_size     = (1024, 1024) # in pixels (width, height)
radius       = 248          # radius of central/inside area
radius_out   = 2 * radius
ring_width   = 20
ring_colour  = (128, 128, 128)

### New approach
Define image files as list in `images`. For each file `some.jpg`, the block will produce two files `in_some.jpg` and `out_some.jpg` with either inside or outside block.

In [3]:
# external ring, inside only and outside only
images  = image_in # <= specify list here

# keep as it is
fn = ["%s%s" % (image_folder, i) for i in images]
fn_gray = "%s%s" % (image_folder, "gray.png")

img = []
for i in fn:
    im = io.imread(i)
    if im.dtype == "O":
        im = im[0]  # ugly hack to overcome old PIL library, pillow should be used http://stackoverflow.com/questions/24089337/skimage-simple-imread-of-image-return-strange-output 
    cropped = crop_and_resize_image(img_as_float(im), new_size, grayscale=True)
    img.extend([cropped])
gray = crop_and_resize_image(img_as_float(io.imread(fn_gray)), new_size, grayscale=True)
_, ring = make_ring_for_image(img[0], radius = radius, width50 = ring_width, colour = ring_colour)
_, ring_out = make_ring_for_image(img[0], radius = radius_out, width50 = ring_width, colour = ring_colour)

for ii in np.arange(len(img)):
    print "image: %d %s" % (ii, images[ii])
    fni  = "output/in_%s" % images[ii]
    im1  = combine_images(img[ii], gray, radius)
    im = combine_images(im1, gray, radius_out)
    if ring_width == 0:
        imf = im
    else:
        imf = add_ring(im, ring)
        imf = add_ring(imf, ring_out)            
    io.imsave(fni, imf)
    # and out
    fno  = "output/out_%s" % images[ii]
    im1  = combine_images(gray, img[ii], radius)
    im = combine_images(im1, gray, radius_out)
    if ring_width == 0:
        imf = im
    else:
        imf = add_ring(im, ring)
        imf = add_ring(imf, ring_out)            
    io.imsave(fno, imf)
    

image: 0 MH1.jpg


/Users/jiri/.conda/envs/npsi/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


image: 1 MH2.jpg
image: 2 MH3.jpg
image: 3 MH4.jpg
image: 4 MH5.jpg
image: 5 MM1.jpg
image: 6 MM2.jpg
image: 7 MM3.jpg
image: 8 MM4.jpg
image: 9 MM5.jpg
image: 10 MP1.jpg
image: 11 MP2.jpg
image: 12 MP3.jpg
image: 13 MP4.jpg
image: 14 MP5.jpg
image: 15 MS1.jpg
image: 16 MS2.jpg
image: 17 MS3.jpg
image: 18 MS4.jpg
image: 19 MS5.jpg
image: 20 MT1.jpg
image: 21 MT2.jpg
image: 22 MT3.jpg
image: 23 MT4.jpg
image: 24 MT5.jpg
image: 25 NB1.jpg
image: 26 NB2.jpg
image: 27 NB3.jpg
image: 28 NB4.jpg
image: 29 NB5.jpg
image: 30 ND1.jpg
image: 31 ND2.jpg
image: 32 ND3.jpg
image: 33 ND4.jpg
image: 34 ND5.jpg
image: 35 NF1.jpg
image: 36 NF2.jpg
image: 37 NF3.jpg
image: 38 NF4.jpg
image: 39 NF5.jpg
image: 40 NM1.jpg
image: 41 NM2.jpg
image: 42 NM3.jpg
image: 43 NM4.jpg
image: 44 NM5.jpg
image: 45 NR1.jpg
image: 46 NR2.jpg
image: 47 NR3.jpg
image: 48 NR4.jpg
image: 49 NR5.jpg


# Finished


### Original approach
With two image lists, producing all possible combinations. Discards original file names, resulting images are called e.g. `001_002.jpeg`.

In [ ]:
# original (without external circle)
fn_in  = ["%s%s" % (image_folder, i) for i in image_in]
fn_out = ["%s%s" % (image_folder, i) for i in image_out]

img_in  = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_in]
img_out = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_out]
_, ring    = make_ring_for_image(img_in[0], radius = radius, width50 = ring_width, colour = ring_colour)

for iin in np.arange(len(image_in)):
    for iout in np.arange(len(image_out)):
        fn  = "output/%03d_%03d.jpeg" % (iin, iout)
        im  = combine_images(img_in[iin], img_out[iout], radius)
        if ring_width == 0:
            imf = im
        else:
            imf = add_ring(im, ring)
        io.imsave(fn, imf)
    

In [ ]:
# same as original, but with external ring
fn_gray = "%s%s" % (image_folder, "gray.png")
fn_in  = ["%s%s" % (image_folder, i) for i in image_in]
fn_out = ["%s%s" % (image_folder, i) for i in image_out]

img_in  = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_in]
img_out = [crop_and_resize_image(
        img_as_float(io.imread(i)), new_size, grayscale = True) for i in fn_out]
gray = crop_and_resize_image(img_as_float(io.imread(fn_gray)), new_size, grayscale=True)
_, ring = make_ring_for_image(img_in[0], radius = radius, width50 = ring_width, colour = ring_colour)
_, ring_out = make_ring_for_image(img_in[0], radius = radius_out, width50 = ring_width, colour = ring_colour)

for iin in np.arange(len(img_in)):
    for iout in np.arange(len(img_out)):
        print "im %d %d" % (iin, iout)
        fn  = "output/%03d_%03d.jpeg" % (iin, iout)
        im1  = combine_images(img_in[iin], img_out[iout], radius)
        im = combine_images(im1, gray, radius_out)
        if ring_width == 0:
            imf = im
        else:
            imf = add_ring(im, ring)
            imf = add_ring(imf, ring_out)            
        io.imsave(fn, imf)
    